In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30471 entries, 0 to 30470
Columns: 292 entries, id to price_doc
dtypes: float64(119), int64(157), object(16)
memory usage: 67.9+ MB


In [5]:
df.head()

,id,timestamp,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,...,cafe_count_5000_price_2500,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc
0,1,2011-08-20,43,27.0,4.0,NaN,NaN,NaN,NaN,NaN,...,9,4,0,13,22,1,0,52,4,5850000
1,2,2011-08-23,34,19.0,3.0,NaN,NaN,NaN,NaN,NaN,...,15,3,0,15,29,1,10,66,14,6000000
2,3,2011-08-27,43,29.0,2.0,NaN,NaN,NaN,NaN,NaN,...,10,3,0,11,27,0,4,67,10,5700000
3,4,2011-09-01,89,50.0,9.0,NaN,NaN,NaN,NaN,NaN,...,11,2,1,4,4,0,0,26,3,13100000
4,5,2011-09-05,77,77.0,4.0,NaN,NaN,NaN,NaN,NaN,...,319,108,17,135,236,2,91,195,14,16331452


In [8]:
missing_cols = df.columns[df.isna().any()].tolist()
missing_cols

In [23]:
cafe_cols = [col for col in missing_cols if 'cafe' in col]
cafe_cols

['cafe_sum_500_min_price_avg',
 'cafe_sum_500_max_price_avg',
 'cafe_avg_price_500',
 'cafe_sum_1000_min_price_avg',
 'cafe_sum_1000_max_price_avg',
 'cafe_avg_price_1000',
 'cafe_sum_1500_min_price_avg',
 'cafe_sum_1500_max_price_avg',
 'cafe_avg_price_1500',
 'cafe_sum_2000_min_price_avg',
 'cafe_sum_2000_max_price_avg',
 'cafe_avg_price_2000',
 'cafe_sum_3000_min_price_avg',
 'cafe_sum_3000_max_price_avg',
 'cafe_avg_price_3000',
 'cafe_sum_5000_min_price_avg',
 'cafe_sum_5000_max_price_avg',
 'cafe_avg_price_5000']

In [36]:

df.groupby('sub_area').cafe_sum_500_min_price_avg.mean().head(50)

sub_area
Ajeroport                  531.283276
Akademicheskoe             944.128436
Alekseevskoe               727.457368
Altuf'evskoe               795.523529
Arbat                     1045.482000
Babushkinskoe              543.628407
Basmannoe                  714.295204
Begovoe                   1000.846000
Beskudnikovskoe            821.249662
Bibirevo                   584.778010
Birjulevo Vostochnoe       759.081667
Birjulevo Zapadnoe         325.609756
Bogorodskoe                620.754717
Brateevo                   416.666667
Butyrskoe                  667.727286
Caricyno                  1050.915714
Cheremushki                818.211042
Chertanovo Central'noe     629.299125
Chertanovo Juzhnoe         477.777544
Chertanovo Severnoe        654.246180
Danilovskoe                768.952434
Dmitrovskoe               1125.324388
Donskoe                    885.999845
Dorogomilovo               985.903929
Filevskij Park             645.892901
Fili Davydkovo             656.013187
Gag

In [37]:
df2 = df.copy()
cafe_cols = [col for col in missing_cols if 'cafe' in col]
cols_fill = cafe_cols.copy()
cols_fill.insert(0,'sub_area')
df2[cafe_cols] = df2[cols_fill].groupby('sub_area').transform(lambda x: x.fillna(x.mean()))
df2['prom_part_5000'] = df2['prom_part_5000'].fillna(df2['prom_part_5000'].mean())

In [34]:
df2[cols_fill]

,sub_area,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000
0,Bibirevo,584.77801,966.940561,775.85949,527.78,888.89,708.33,566.67,969.70,768.18,614.29,1042.86,828.57,639.68,1079.37,859.52,708.57,1185.71,947.14
1,Nagatinskij Zaton,860.00000,1500.000000,1180.00000,615.38,1076.92,846.15,694.12,1205.88,950.00,695.24,1190.48,942.86,631.03,1086.21,858.62,673.81,1148.81,911.31
2,Tekstil'shhiki,666.67000,1166.670000,916.67000,642.86,1142.86,892.86,516.67,916.67,716.67,563.64,977.27,770.45,697.44,1192.31,944.87,702.68,1196.43,949.55
3,Mitino,1000.00000,1500.000000,1250.00000,658.33,1083.33,870.83,673.91,1130.43,902.17,660.00,1120.00,890.00,718.75,1218.75,968.75,931.58,1552.63,1242.11
4,Basmannoe,702.22000,1166.670000,934.44000,763.45,1272.41,1017.93,766.80,1272.73,1019.76,765.93,1269.23,1017.58,853.03,1410.45,1131.74,853.88,1411.45,1132.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,Otradnoe,614.29000,1071.430000,842.86000,666.67,1138.89,902.78,646.15,1115.38,880.77,635.90,1102.56,869.23,647.89,1112.68,880.28,689.95,1156.08,923.02
30467,Tverskoe,777.42000,1322.580000,1050.00000,699.35,1183.01,941.18,762.18,1269.09,1015.64,796.39,1321.69,1059.04,895.84,1476.65,1186.24,887.43,1462.88,1175.16
30468,Poselenie Vnukovskoe,1000.00000,1750.000000,1375.00000,1000.00,1750.00,1375.00,833.33,1500.00,1166.67,757.14,1285.71,1021.43,700.00,1222.22,961.11,747.37,1263.16,1005.26
30469,Obruchevskoe,1000.00000,1500.000000,1250.00000,753.85,1269.23,1011.54,646.34,1097.56,871.95,704.69,1195.31,950.00,681.48,1152.78,917.13,703.20,1182.65,942.92


In [35]:
df[cols_fill]

,sub_area,cafe_sum_500_min_price_avg,cafe_sum_500_max_price_avg,cafe_avg_price_500,cafe_sum_1000_min_price_avg,cafe_sum_1000_max_price_avg,cafe_avg_price_1000,cafe_sum_1500_min_price_avg,cafe_sum_1500_max_price_avg,cafe_avg_price_1500,cafe_sum_2000_min_price_avg,cafe_sum_2000_max_price_avg,cafe_avg_price_2000,cafe_sum_3000_min_price_avg,cafe_sum_3000_max_price_avg,cafe_avg_price_3000,cafe_sum_5000_min_price_avg,cafe_sum_5000_max_price_avg,cafe_avg_price_5000
0,Bibirevo,NaN,NaN,NaN,527.78,888.89,708.33,566.67,969.70,768.18,614.29,1042.86,828.57,639.68,1079.37,859.52,708.57,1185.71,947.14
1,Nagatinskij Zaton,860.00,1500.00,1180.00,615.38,1076.92,846.15,694.12,1205.88,950.00,695.24,1190.48,942.86,631.03,1086.21,858.62,673.81,1148.81,911.31
2,Tekstil'shhiki,666.67,1166.67,916.67,642.86,1142.86,892.86,516.67,916.67,716.67,563.64,977.27,770.45,697.44,1192.31,944.87,702.68,1196.43,949.55
3,Mitino,1000.00,1500.00,1250.00,658.33,1083.33,870.83,673.91,1130.43,902.17,660.00,1120.00,890.00,718.75,1218.75,968.75,931.58,1552.63,1242.11
4,Basmannoe,702.22,1166.67,934.44,763.45,1272.41,1017.93,766.80,1272.73,1019.76,765.93,1269.23,1017.58,853.03,1410.45,1131.74,853.88,1411.45,1132.66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30466,Otradnoe,614.29,1071.43,842.86,666.67,1138.89,902.78,646.15,1115.38,880.77,635.90,1102.56,869.23,647.89,1112.68,880.28,689.95,1156.08,923.02
30467,Tverskoe,777.42,1322.58,1050.00,699.35,1183.01,941.18,762.18,1269.09,1015.64,796.39,1321.69,1059.04,895.84,1476.65,1186.24,887.43,1462.88,1175.16
30468,Poselenie Vnukovskoe,1000.00,1750.00,1375.00,1000.00,1750.00,1375.00,833.33,1500.00,1166.67,757.14,1285.71,1021.43,700.00,1222.22,961.11,747.37,1263.16,1005.26
30469,Obruchevskoe,1000.00,1500.00,1250.00,753.85,1269.23,1011.54,646.34,1097.56,871.95,704.69,1195.31,950.00,681.48,1152.78,917.13,703.20,1182.65,942.92
